# Working with parquet files - Maksym Parkhomchuk

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
%load_ext dotenv
%dotenv


In [3]:
import pandas as pd
import yfinance as yf
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
from dotenv import load_dotenv
load_dotenv()
folder = os.getenv('SRC_DIR')
print(folder)

../../05_src/


In [4]:
import dask.dataframe as dd

/var/folders/05/5ck2p97d4jj5tx1lz6c4j7980000gn/T/ipykernel_14200/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [7]:
import os
from glob import glob

PRICE_DATA = os.getenv('PRICE_DATA')
# Use glob to get all parquet files (recursively)
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [8]:
dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        # Add lag for 'Close'
        Close_lag = x['Close'].shift(1),
        
        # Add lag for 'Adj Close'
        Adj_Close_lag = x['Adj Close'].shift(1),
    )
)

# Now calculate the 'returns' and 'hi_lo_range' after the lag columns are added
dd_feat = dd_feat.assign(
    returns = (dd_feat['Adj Close'] / dd_feat['Adj_Close_lag']) - 1,
    hi_lo_range = dd_feat['High'] - dd_feat['Low']
)


/var/folders/05/5ck2p97d4jj5tx1lz6c4j7980000gn/T/ipykernel_14200/2330299757.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [9]:
# Convert Dask DataFrame to Pandas DataFrame
dd_feat_pandas = dd_feat.compute()
dd_feat_pandas = dd_feat_pandas.sort_values(by=['ticker', 'Date'])


# Calculate the rolling average return with a 10-day window
dd_feat_pandas['rolling_avg_return'] = dd_feat_pandas.groupby('ticker')['returns'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

# Verify the result
print(dd_feat_pandas.head())



             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.463039   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.142941   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.652878   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.219193   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.237453   

         Volume       sector                       subsector  year  Close_lag  \
ticker                                                                          
A       4674353  Health Care  Life Sciences Tools & Services  2000        NaN   
A       4765083  Health Care  Life Sciences Tools & Services  2000  51.502148   
A       5758642  Health Care  Life Sciences Tools & Services  2000  47.567955   
A       2534434  Health Care  Life Sciences Tools & Services  

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

It wasn't necessary, since a moving average return can be calculated in dask. However, groupby operation and rolling methods in dask require careful handling

+ Would it have been better to do it in Dask? Why?

It would generally have been better to calculate the rolling average return in Dask, particularly for large datasets, since it would run calculations in parallel and would optimize memory use.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.